import pandas as pd

In [1]:
import pandas as pd

In [2]:
tripadv_review = pd.read_csv(r'C:\Users\Dell\Desktop\Hotel\Europa Plaza Hotel.csv', error_bad_lines=False);

In [3]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)

In [4]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(token)
    return result

In [6]:
processed_reviews = tripadv_review['r_text'].map(preprocess)
processed_reviews

0     [nice, people, helpful, breakfast, good, room,...
1     [stay, hotel, like, loud, music, noisy, traffi...
2     [waste, money, rude, receptionist, want, exper...
3     [things, report, hotel, unfortunately, late, n...
4     [stayed, nights, early, october, located, clos...
                            ...                        
68    [europa, quiet, clean, friendly, hotel, minute...
69    [stay, hotel, days, augustvery, frienfly, staf...
70    [beautiful, experience, nice, clean, large, ro...
71    [nice, clean, modern, hotel, aimed, mainly, bu...
72    [stayed, europa, nights, business, pleasant, n...
Name: r_text, Length: 73, dtype: object

In [7]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(processed_reviews)

# Create Corpus
texts = processed_reviews

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1)]]


In [9]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=50,
                                           update_every=1,
                                           chunksize=100,
                                           passes=5,
                                           alpha='auto',
                                           eta='auto',
                                           per_word_topics=True,
                                           iterations = 25,
                                           eval_every = None)

In [10]:
from pprint import pprint

In [11]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.010*"room" + 0.007*"booking" + 0.007*"towels" + 0.007*"smoking" + '
  '0.006*"quiet" + 0.005*"rooms" + 0.004*"twin" + 0.004*"hotel" + '
  '0.004*"absolutely" + 0.004*"dirty"'),
 (1,
  '0.030*"room" + 0.026*"hotel" + 0.021*"good" + 0.011*"night" + '
  '0.010*"breakfast" + 0.009*"clean" + 0.008*"receptionist" + 0.008*"rooms" + '
  '0.007*"coffee" + 0.007*"asked"'),
 (2,
  '0.036*"hotel" + 0.017*"room" + 0.014*"night" + 0.012*"clean" + 0.012*"nice" '
  '+ 0.011*"staff" + 0.010*"breakfast" + 0.009*"rooms" + 0.008*"food" + '
  '0.008*"good"'),
 (3,
  '0.037*"hotel" + 0.021*"good" + 0.017*"breakfast" + 0.015*"clean" + '
  '0.014*"rooms" + 0.014*"room" + 0.014*"nicosia" + 0.013*"staff" + '
  '0.011*"location" + 0.010*"business"'),
 (4,
  '0.022*"hotel" + 0.012*"noise" + 0.011*"good" + 0.011*"breakfast" + '
  '0.011*"rooms" + 0.011*"staff" + 0.010*"room" + 0.009*"great" + '
  '0.009*"helpful" + 0.008*"clean"')]


In [12]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.063702 -0.083227       1        1  34.163925
1      0.094137 -0.034967       2        1  27.356085
2     -0.032979  0.063862       3        1  21.938015
4     -0.026896  0.026688       4        1  14.883837
0      0.029440  0.027644       5        1   1.658137, topic_info=        Term       Freq      Total Category  logprob  loglift
12      room  53.000000  53.000000  Default  30.0000  30.0000
235    quiet   4.000000   4.000000  Default  29.0000  29.0000
129   towels   3.000000   3.000000  Default  28.0000  28.0000
9      noise  13.000000  13.000000  Default  27.0000  27.0000
7    helpful  14.000000  14.000000  Default  26.0000  26.0000
..       ...        ...        ...      ...      ...      ...
129   towels   0.338694   3.379564   Topic5  -4.9768   1.7991
235    quiet   0.294205   4.256876   Topic5  -5.1176   1.4275
12      room   0.482657  53.701199   Topic5  -4.6226  -0.6124
13     rooms   0.251308  32.041252   Topic5  -5.2752  -0.7486
20     hotel   0.205740  91.929100   Topic5  -5.4753  -2.0027

[328 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
117       3  0.659119  absolutely
527       3  0.710014  acceptable
952       4  0.767627     adapter
965       1  0.874057       ample
759       4  0.772383   apartment
...     ...       ...         ...
101       4  0.121975        work
546       2  0.935176      worked
38        3  0.716737       worst
752       1  0.872055       worth
116       3  0.739192      worths

[422 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 2, 3, 5, 1])